In [72]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
import time


In [73]:
url_ = "https://www.flipkart.com/"
driver = webdriver.Chrome()
driver.get(url_)
driver.maximize_window()

In [74]:
categories= {
    "Electronics": ["Smart Watches"],
    "Fashion": ["Men's Clothing", "Women's Clothing", "Kids' Clothing", "Footwear", "Watches", "Sunglasses", "Handbags", "Jewelry"],
    "Home and Furniture": ["Storage"],
    "Travel and Luggage": ["Suitcases", "Backpacks", "Duffel Bags", "Travel Accessories", "Wallets", "Belts", "Trolley Bags", "Luggage Sets"]
}

d = {"Product":[],"Price":[],"Original Price":[],"Discount":[],"Rating":[],"Rating & reviews":[], "Category": [], "Sub category": [], "brand": []}

In [75]:
wait = WebDriverWait(driver, 10)

In [76]:
def scrap_current_page(c,s):
    try:
        elem = driver.find_elements(By.XPATH, "//div[@class='_1sdMkc LFEi7Z']")   
        for i in range(len(elem)):
            d["Category"].append(c)
            d["Sub category"].append(s)
            try:
                elem[i].find_element(By.CLASS_NAME,"WKTcLC.BwBZTg")
                elem[i].find_element(By.CLASS_NAME,"syl9yP")
                d['Product'].append(elem[i].find_element(By.CLASS_NAME,"WKTcLC.BwBZTg").text)
                d['brand'].append(elem[i].find_element(By.CLASS_NAME,"syl9yP").text)
            except:
                try: 
                    elem[i].find_element(By.CLASS_NAME,"WKTcLC")
                    elem[i].find_element(By.CLASS_NAME,"syl9yP")
                    d['Product'].append(elem[i].find_element(By.CLASS_NAME,"WKTcLC").text)
                    d['brand'].append(elem[i].find_element(By.CLASS_NAME,"syl9yP").text)
                except:
                    elem[i].find_element(By.CLASS_NAME,"WKTcLC")
                    p_name = elem[i].find_element(By.CLASS_NAME,"WKTcLC").text
                    d['Product'].append(p_name)
                    brand = p_name.split(" ")[0]
                    d['brand'].append(brand)
            try:
                d['Price'].append(elem[i].find_element(By.CLASS_NAME,"Nx9bqj").text)
            except:
                d["Price"].append(np.nan)
            try:
                d['Original Price'].append(elem[i].find_element(By.CLASS_NAME,"yRaY8j").text)
            except:
                d["Original Price"].append(np.nan)
            try:
                d['Discount'].append(elem[i].find_element(By.CLASS_NAME,"UkUFwK").text)
            except:
                d["Discount"].append(np.nan)
            try:
                d['Rating'].append(elem[i].find_element(By.CLASS_NAME,"XQDdHH").text)
            except:
                d["Rating"].append(np.nan)
            try:
                d['Rating & reviews'].append(elem[i].find_element(By.CLASS_NAME, "Wphh3N").text)
            except:
                d["Rating & reviews"].append(np.nan)
            
    except Exception as e:
        print("Error loading cards:", e)

In [77]:
def full_scrape(c,s):
    i=1
    # while True:
    url = f"https://www.flipkart.com/search?q={s}&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={i}"
    try:
        driver.get(url)
        if len(driver.find_elements(By.XPATH, "//div[@class='_1sdMkc LFEi7Z']")) > 0: 
            scrap_current_page(c,s)
            print(s)
            i+=1
        else:
            print(f"Terminated{s}") 
            # break
    except Exception as e:
        print("No more pages or error in pagination:", e)
        # break

In [78]:
for c in categories:
    for s in categories[c]:
        search = driver.find_element(By.XPATH, "//input[@placeholder='Search for Products, Brands and More']")
        search.send_keys(s)
        search.send_keys(Keys.RETURN)
        time.sleep(2)
        full_scrape(c,s)
        driver.get(url_)

Smart Watches
Men's Clothing
Women's Clothing
Kids' Clothing
Footwear
Watches
Sunglasses
Handbags
Jewelry
Storage
Suitcases
Backpacks
Duffel Bags
Travel Accessories
Wallets
Belts
Trolley Bags
Luggage Sets


In [79]:
import pandas as pd
df = pd.DataFrame(d)
df

,Product,Price,Original Price,Discount,Rating,Rating & reviews,Category,Sub category,brand
0,"Boult Drift+ Bluetooth Calling, 1.85"" HD, 500N...","₹1,399","₹8,499",83% off,NaN,NaN,Electronics,Smart Watches,Boult
1,"Boult Drift BT Calling HD Display, 140+ Watchf...","₹1,299","₹7,999",83% off,NaN,NaN,Electronics,Smart Watches,Boult
2,Fastrack Revoltt FS1 1.83 Display|BT Calling|F...,"₹1,099","₹3,995",72% off,NaN,NaN,Electronics,Smart Watches,Fastrack
3,"Gamesir T800 BT watch 1.83"" Screen Sunlight Pr...",₹617,"₹1,999",69% off,NaN,NaN,Electronics,Smart Watches,Gamesir
4,Fastrack Revoltt FS1|1.83 Display|BT Calling|F...,"₹1,099","₹3,995",72% off,NaN,NaN,Electronics,Smart Watches,Fastrack
...,...,...,...,...,...,...,...,...,...
715,Hard Body Set of 2 Luggage 8 Wheels - Paris Ha...,"₹8,049","₹33,990",76% off,NaN,NaN,Travel and Luggage,Luggage Sets,NASHER MILES
716,Soft Body Set of 6 Luggage 4 Wheels - Trigon H...,"₹5,999","₹42,080",85% off,NaN,NaN,Travel and Luggage,Luggage Sets,ARISTOCRAT
717,Hard Body Set of 2 Luggage 4 Wheels - ZENO SET...,"₹4,049","₹19,455",79% off,NaN,NaN,Travel and Luggage,Luggage Sets,SAFARI
718,Hard Body Set of 3 Luggage 8 Wheels - Nicobar ...,"₹8,949","₹37,985",76% off,NaN,NaN,Travel and Luggage,Luggage Sets,NASHER MILES


In [80]:
df.isnull().sum()

Product               0
Price                 0
Original Price       17
Discount             17
Rating              720
Rating & reviews    720
Category              0
Sub category          0
brand                 0
dtype: int64

In [81]:
# df.to_csv("Flipkart_sabkuch.csv")